In [102]:
import pandas as pd

df = pd.read_excel('Ramazan 2024 v1.xlsx')

In [103]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Zakat,Unnamed: 6,Sadqah,Unnamed: 8,Total,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Name,Held By,Donor / Vendor,Date,Mode of Transaction,Collection,Balance,Collection,Balance,Collection,Balance,NaN,NaN,Zakaat,Sadqah,Total,NaN,Accrual
1,Mughees,Mughees,Opening Balance,2025-03-01 00:00:00,Cash,46660,46660,29656,29656,76316,76316,NaN,Usman,0,0,0,NaN,0
2,Uraib,Uraib,Opening Balance,2025-03-01 00:00:00,Cash,26200,72860,8158,37814,34358,110674,NaN,Mughees,169036,-11300,157736,NaN,0
3,Maaiz,Maaiz,Opening Balance,2025-03-01 00:00:00,Cash,-100690,-27830,252930,290744,152240,262914,NaN,Hammad,0,0,0,NaN,0
4,Bilal,Bilal,Opening Balance,2025-03-01 00:00:00,Cash,2001,-25829,-10400,280344,-8399,254515,NaN,Aleem,515500,298617,814117,NaN,810000


# Single variable things: Referrals, Collectors, Projects

In [104]:
referrals = df.iloc[1:, 0].unique().tolist()
referrals.pop()
referrals

df_referrals = pd.DataFrame(data={"Referrals": referrals})
df_referrals.to_csv('seed_tables_data/referrals.csv', index=True)

In [105]:
collectors = df.iloc[1:, 1].unique().tolist()
# collectors.remove('nan')
collectors.pop()
collectors
df_collectors = pd.DataFrame(data={"Collectors": collectors})
df_collectors.to_csv('seed_tables_data/collectors.csv', index=True)

In [106]:
projects = df.loc[df.iloc[:, 7].astype(str).str.startswith("-") | df.iloc[:, 5].astype(str).str.startswith("-"), df.columns[2]].unique().tolist()
# len(projects)
df_projects = pd.DataFrame(data={"Projects": projects})
df_projects.to_csv('seed_tables_data/projects.csv', index=True)

# Tables: Donations, Expenses, Payments

In [107]:
methods = df.iloc[1:, 4].unique().tolist()
methods

['Cash', 'Sadapay', 'MB', 'UBL', 'FB', 'HMB', 'Accrual', nan]

In [108]:
// Expenses table
model Expenses {
  transacId     Int              @id @default(autoincrement())
  date          DateTime
  amount        Int
  paymentMethod String
  vendorProjId  Int
  category      String
  description   String?
  status        String
  
  vendorProject VendorsProjects? @relation(fields: [vendorProjId], references: [vendProjId], onDelete: Cascade)
  payments      Payment[]
}


SyntaxError: invalid syntax (1626130979.py, line 1)

In [ ]:
expenses_sadqa = df.loc[df.iloc[:, 7].astype(str).str.startswith("-"), [df.columns[i] for i in [3, 1, 7, 4, 2]]]
                        #  ["Date", "Amount", "Mode of Transaction", "Donor/Vendor"]] # category will be set as Program, description empty, status as paid
# expenses_zakat = df.loc[df.iloc[:, 9].astype(str).str.startswith("-"), ["Date", "Amount", "Mode of Transaction", "Donor/Vendor"]]
expenses_sadqa.columns = ["date", "collector", "amount", "paymentMethod", "vendorProj"]
expenses_sadqa["collectorId"] = expenses_sadqa["collector"].map(lambda x: collectors.index(x) + 1)
expenses_sadqa["amount"] = -expenses_sadqa["amount"]
expenses_sadqa["paymentMethod"] = expenses_sadqa["paymentMethod"].map(lambda x: x if x == "Cash" else "Online")
expenses_sadqa["category"] = "Program"
expenses_sadqa["description"] = ""
expenses_sadqa["status"] = "Paid"
expenses_sadqa["vendorProjId"] = expenses_sadqa["vendorProj"].map(lambda x: projects.index(x) + 1)
expenses_sadqa["type"] = "Sadqa"
expenses_sadqa.drop(columns=["vendorProj", "collector"], inplace=True)
expenses_sadqa.reset_index(drop=True, inplace=True)
expenses_sadqa

,date,amount,paymentMethod,collectorId,category,description,status,vendorProjId,type
0,2025-03-01 00:00:00,10400,Cash,4,Program,,Paid,1,Sadqa
1,2025-03-01 00:00:00,500,Cash,5,Program,,Paid,1,Sadqa
2,2025-03-05 00:00:00,5000,Online,1,Program,,Paid,4,Sadqa
3,2025-03-05 00:00:00,12200,Online,1,Program,,Paid,5,Sadqa
4,2025-03-05 00:00:00,5000,Cash,3,Program,,Paid,6,Sadqa
...,...,...,...,...,...,...,...,...,...
74,2025-05-10 00:00:00,10000,Cash,3,Program,,Paid,17,Sadqa
75,2025-05-17 00:00:00,20000,Cash,3,Program,,Paid,48,Sadqa
76,2025-05-19 00:00:00,10000,Cash,3,Program,,Paid,49,Sadqa
77,2025-05-19 00:00:00,10000,Cash,3,Program,,Paid,50,Sadqa


In [ ]:
expenses_zakat = df.loc[df.iloc[:, 5].astype(str).str.startswith("-"), [df.columns[i] for i in [3, 1, 5, 4, 2]]]
                        #  ["Date", "Amount", "Mode of Transaction", "Donor/Vendor"]] # category will be set as Program, description empty, status as paid
# expenses_zakat = df.loc[df.iloc[:, 9].astype(str).str.startswith("-"), ["Date", "Amount", "Mode of Transaction", "Donor/Vendor"]]
expenses_zakat.columns = ["date", "collector", "amount", "paymentMethod", "vendorProj"]
expenses_zakat["collectorId"] = expenses_zakat["collector"].map(lambda x: collectors.index(x) + 1)
expenses_zakat["amount"] = -expenses_zakat["amount"]
expenses_zakat["paymentMethod"] = expenses_zakat["paymentMethod"].map(lambda x: x if x == "Cash" else "Online")
expenses_zakat["category"] = "Program"
expenses_zakat["description"] = ""
expenses_zakat["status"] = "Paid"
expenses_zakat["vendorProjId"] = expenses_zakat["vendorProj"].map(lambda x: projects.index(x) + 1)
expenses_zakat["type"] = "Zakat"
expenses_zakat.drop(columns=["vendorProj", "collector"], inplace=True)
expenses_zakat.reset_index(drop=True, inplace=True)
expenses_zakat

,date,amount,paymentMethod,collectorId,category,description,status,vendorProjId,type
0,2025-03-01 00:00:00,100690,Cash,3,Program,,Paid,1,Zakat
1,2025-03-06 00:00:00,1100000,Online,1,Program,,Paid,2,Zakat
2,2025-03-05 00:00:00,10000,Cash,2,Program,,Paid,3,Zakat
3,2025-03-18 00:00:00,10000,Online,3,Program,,Paid,17,Zakat
4,2025-03-18 00:00:00,30000,Online,3,Program,,Paid,18,Zakat
5,2025-03-18 00:00:00,9000,Online,3,Program,,Paid,17,Zakat
6,2025-03-23 00:00:00,590000,Cash,6,Program,,Paid,3,Zakat
7,2025-03-25 00:00:00,475100,Cash,6,Program,,Paid,3,Zakat
8,2025-03-28 00:00:00,5000,Cash,3,Program,,Paid,23,Zakat
9,2025-03-28 00:00:00,7000,Cash,3,Program,,Paid,24,Zakat


In [ ]:
expenses_df = pd.concat([expenses_sadqa, expenses_zakat], ignore_index=True)
expenses_df

,date,amount,paymentMethod,collectorId,category,description,status,vendorProjId,type
0,2025-03-01 00:00:00,10400,Cash,4,Program,,Paid,1,Sadqa
1,2025-03-01 00:00:00,500,Cash,5,Program,,Paid,1,Sadqa
2,2025-03-05 00:00:00,5000,Online,1,Program,,Paid,4,Sadqa
3,2025-03-05 00:00:00,12200,Online,1,Program,,Paid,5,Sadqa
4,2025-03-05 00:00:00,5000,Cash,3,Program,,Paid,6,Sadqa
...,...,...,...,...,...,...,...,...,...
122,2025-05-09 00:00:00,100000,Cash,1,Program,,Paid,44,Zakat
123,2025-05-10 00:00:00,300000,Cash,3,Program,,Paid,2,Zakat
124,2025-05-12 00:00:00,5000,Cash,1,Program,,Paid,35,Zakat
125,2025-05-19 00:00:00,300000,Cash,3,Program,,Paid,2,Zakat


In [ ]:
expenses_table = expenses_df[["date", "amount", "paymentMethod", "vendorProjId", "category", "description", "status"]]
expenses_table.to_csv('seed_tables_data/expenses.csv', index=True)

In [ ]:
// Payment table
model Payment {
  paymentId      Int         @id @default(autoincrement())
  expenseId      Int?
  collectorId    Int
  type           String
  amount         Int
  
  collector      Collectors? @relation(fields: [collectorId], references: [collectorId], onDelete: Cascade)
  expense        Expenses?   @relation(fields: [expenseId], references: [transacId], onDelete: Cascade)
}

In [ ]:
payments_table = expenses_df[["collectorId", "type", "amount"]]
payments_table["expenseId"] = expenses_df.index + 1
payments_table.to_csv('seed_tables_data/payments.csv', index=True)

/tmp/ipykernel_82245/3084766092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payments_table["expenseId"] = expenses_df.index + 1


In [ ]:
// Donation table
model Donation {
  transacId     Int       @id @default(autoincrement())
  date          DateTime
  amount        Int
  paymentMethod String
  donorName     String
  referralId    Int
  collectorId   Int
  type          String
  status        String
  notes         String?

  referral      Referrals?  @relation(fields: [referralId], references: [referralId], onDelete: Cascade)
  collector     Collectors? @relation(fields: [collectorId], references: [collectorId], onDelete: Cascade)
}

In [ ]:
donations_sadqa = df.loc[df.iloc[:, 7].astype(str) > "0", [df.columns[i] for i in [3, 0, 1, 7, 4, 2]]].dropna()
donations_sadqa.drop(index=0, inplace=True)  # drop header row
donations_sadqa.columns = ["date", "referral", "collector", "amount", "paymentMethod", "donorName"]
donations_sadqa["collectorId"] = donations_sadqa["collector"].map(lambda x: collectors.index(x) + 1)
donations_sadqa["referralId"] = donations_sadqa["referral"].map(lambda x: referrals.index(x) + 1)
donations_sadqa["paymentMethod"] = donations_sadqa["paymentMethod"].map(lambda x: x if x == "Cash" else "Online")
donations_sadqa["status"] = "Completed"
donations_sadqa["type"] = "Sadqa"
donations_sadqa.drop(columns=["collector", "referral"], inplace=True)
donations_sadqa.reset_index(drop=True, inplace=True)
donations_sadqa


,date,amount,paymentMethod,donorName,collectorId,referralId,status,type
0,2025-03-01 00:00:00,29656,Cash,Opening Balance,1,1,Completed,Sadqa
1,2025-03-01 00:00:00,8158,Cash,Opening Balance,2,2,Completed,Sadqa
2,2025-03-01 00:00:00,252930,Cash,Opening Balance,3,3,Completed,Sadqa
3,2025-02-26 00:00:00,150000,Online,Swaleha Uzair,3,3,Completed,Sadqa
4,2025-03-01 00:00:00,5000,Online,Sufyan Kaleem,1,1,Completed,Sadqa
...,...,...,...,...,...,...,...,...
164,2025-04-06 00:00:00,500,Cash,Transfer,5,9,Completed,Sadqa
165,2025-04-08 00:00:00,20000,Cash,Iqbal Chamdia,3,3,Completed,Sadqa
166,2025-04-06 00:00:00,25000,Cash,Transfer,3,9,Completed,Sadqa
167,2025-04-06 00:00:00,137124,Cash,Transfer,1,9,Completed,Sadqa


In [ ]:
donations_zakat = df.loc[df.iloc[:, 5].astype(str) > "0", [df.columns[i] for i in [3, 0, 1, 5, 4, 2]]].dropna()
donations_zakat.drop(index=0, inplace=True)  # drop header row
donations_zakat.columns = ["date", "referral", "collector", "amount", "paymentMethod", "donorName"]
donations_zakat["collectorId"] = donations_zakat["collector"].map(lambda x: collectors.index(x) + 1)
donations_zakat["referralId"] = donations_zakat["referral"].map(lambda x: referrals.index(x) + 1)
donations_zakat["paymentMethod"] = donations_zakat["paymentMethod"].map(lambda x: x if x == "Cash" else "Online")
donations_zakat["status"] = "Completed"
donations_zakat["type"] = "Zakat"
donations_zakat.drop(columns=["collector", "referral"], inplace=True)
donations_zakat.reset_index(drop=True, inplace=True)
donations_zakat

,date,amount,paymentMethod,donorName,collectorId,referralId,status,type
0,2025-03-01 00:00:00,46660,Cash,Opening Balance,1,1,Completed,Zakat
1,2025-03-01 00:00:00,26200,Cash,Opening Balance,2,2,Completed,Zakat
2,2025-03-01 00:00:00,2001,Cash,Opening Balance,4,4,Completed,Zakat
3,2025-03-01 00:00:00,2000,Cash,Opening Balance,5,5,Completed,Zakat
4,2025-02-27 00:00:00,55000,Online,Haji Zubair,1,1,Completed,Zakat
...,...,...,...,...,...,...,...,...
126,2025-04-27 00:00:00,20000,Cash,Transfer,4,9,Completed,Zakat
127,2025-04-30 00:00:00,2000,Cash,Asba Imran,1,9,Completed,Zakat
128,2025-04-30 00:00:00,5000,Cash,Huzaifa Mahenti,1,9,Completed,Zakat
129,2025-04-30 00:00:00,287506,Cash,Abdul Samad Nara,1,9,Completed,Zakat


In [ ]:
donations_df = pd.concat([donations_sadqa, donations_zakat], ignore_index=True)
donations_df.to_csv('seed_tables_data/donations.csv', index=False)
donations_df

,date,amount,paymentMethod,donorName,collectorId,referralId,status,type
0,2025-03-01 00:00:00,29656,Cash,Opening Balance,1,1,Completed,Sadqa
1,2025-03-01 00:00:00,8158,Cash,Opening Balance,2,2,Completed,Sadqa
2,2025-03-01 00:00:00,252930,Cash,Opening Balance,3,3,Completed,Sadqa
3,2025-02-26 00:00:00,150000,Online,Swaleha Uzair,3,3,Completed,Sadqa
4,2025-03-01 00:00:00,5000,Online,Sufyan Kaleem,1,1,Completed,Sadqa
...,...,...,...,...,...,...,...,...
295,2025-04-27 00:00:00,20000,Cash,Transfer,4,9,Completed,Zakat
296,2025-04-30 00:00:00,2000,Cash,Asba Imran,1,9,Completed,Zakat
297,2025-04-30 00:00:00,5000,Cash,Huzaifa Mahenti,1,9,Completed,Zakat
298,2025-04-30 00:00:00,287506,Cash,Abdul Samad Nara,1,9,Completed,Zakat
